# Quantum Process Tomography

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
A central challenge on the path towards large-scale quantum computing is the engineering of high-quality quantum process. A method that accurately and reliably characterize unknown quantum process $\mathcal{N}$ is desirable.

Quantum Process Tomography (QPT) deals with identifying an unknown quantum dynamical process. It requires the use of quantum state tomography to reconstruct the process.

This tutorial introduces Quantum Process Tomography (QPT), covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory


### Pauli transfer matrix representation
The Pauli Transfer Matrix (PTM) representation of an $n$-qubit quantum channel $\mathcal{N}$ is defined with respect to vectorization in Pauli basis instead of column-vectorization. The elements of the PTM of $[\mathcal{N}]$ are given by

$$
    [\mathcal{N}]_{ij} = \textrm{Tr} \left[ P_i \mathcal{N} (P_j) \right],
$$

where

$$
    P_i, P_j \in \left\{ I / \sqrt{2}, X / \sqrt{2},  Y / \sqrt{2},  Z / \sqrt{2} \right\}^{\otimes n}.
$$

Let $\textrm{Tr}\left[ P_i \mathcal{N} (P_j) \right] = \langle\langle i | [\mathcal{N}] | j \rangle\rangle$, where $|i\rangle,$ and $|j\rangle$ are the bases corresponding to $P_i$ and $P_j$ in the PTM representation. Under this notation, a quantum process can be represented as a $4^n \times 4^n$ square matrix

$$
\begin{align}
    [\mathcal{N}] &= \left( \sum_i | i \rangle\rangle \langle\langle i | \right) [\mathcal{N}] \left( \sum_j | j \rangle\rangle \langle\langle j | \right) \\
    &= \sum_{ij} | i \rangle\rangle \langle\langle i | [\mathcal{N}]| j \rangle\rangle \langle\langle j |.
\end{align}
$$



### Process tomography method

Similar to QST, we can characterize the quantum process $\mathcal{N}$ by measuring its Pauli transfer matrix $[\mathcal{N}]$. To obtain these components, it is usually more experimentally convenient to use a different basis of measurement operators $\left\{ E_m \right\}_{m=1}^M(M \geq d^2)$ and preparation operators $\left\{ \rho_n \right\}_{n=1}^N(N \geq d^2)$ that both span the Hilbert-Schmidt space $\mathscr{L}(\mathcal{H}_d)$, where $d = 2^n$ and $n$ is the number of qubits. Generally, one measures the $MN$ expectation values (or you may say probabilities)

$$
    \forall m \in [M], n \in [N], p_{mn} := \langle\langle E_m | [\mathcal{N}] | \rho_n \rangle\rangle = \textrm{Tr} \left[ E_m \mathcal{N}(\rho_n) \right].
$$

Inserting the complete set of Pauli basis, we have

$$
\begin{align}
    p_{mn} &= \langle\langle E_m | [\mathcal{N}] | \rho_n \rangle\rangle \\
    &= \langle\langle E_m | \left( \sum_{j=1}^{d^2} | j \rangle\rangle \langle\langle j | \right) [\mathcal{N}] \left( \sum_{k=1}^{d^2} | k \rangle\rangle \langle\langle k | \right) | \rho_n \rangle\rangle \\
    &= \sum_{jk} \langle\langle E_m | j \rangle\rangle \langle\langle j | [\mathcal{N}] | k \rangle\rangle \langle\langle k | \rho_n \rangle\rangle,
\end{align}
$$

where $\langle\langle E_m | j \rangle\rangle$ are the $j$-th element of $E_m$ expanding in the Pauli basis and similarly for $\langle\langle k | \rho_n \rangle\rangle$.

Since the basis $\left\{ E_m \right\}$ is chosen in advance by the experimenter, we can define an $M \times d^2$ dimension matrix $\mathfrak{M}$,

$$
\begin{align}
  \mathfrak{M} = \begin{pmatrix}
    \langle \langle E_1 | 1 \rangle \rangle & \langle \langle E_1 | 2 \rangle \rangle & \cdots & \langle \langle E_1 | d^2 \rangle \rangle \\
    \langle \langle E_2 | 1 \rangle \rangle & \langle \langle E_2 | 2 \rangle \rangle & \cdots & \langle \langle E_2 | d^2 \rangle \rangle \\
    \vdots & \vdots & \ddots & \vdots \\
    \langle \langle E_M | 1 \rangle \rangle & \langle \langle E_M | 2 \rangle \rangle & \cdots & \langle \langle E_M | d^2 \rangle \rangle \\
  \end{pmatrix}.
\end{align}
$$

Similar to QST, we may indirectly estimate $[\mathcal{N}]$ by approaching the expectation values $p_{mn}$ experimentally. Indeed, they can be experimentally estimated by measuring many copies of the state $\rho_n$ and calculate the corresponding expectation value following the description in quantum state tomography, as shown in the figure below.
 
![QPT](./figures/qpt-circuit.png "Figure 1: We construct such circuits to estimate expectation values. ")
 
The equation can also be written in matrix form as

$$
\begin{align}
  \mathbf{P} = \mathfrak{M} \left[\mathcal{N}\right] \mathfrak{P}^T,
\end{align}
$$

where $P_{mn} = p_{mn}$.
Similarly, we can use linear inversion estimation or ordinary least squares to estimate $[\mathcal{N}]$.
Finally, the flow chart is as follows.
 
![QPT](./figures/qpt-chart.png "Figure 2: We complete QPT according to this flow chart. ")
 

## Practice

We demonstrate quantum process tomography on the CNOT gate.

First, we import the necessary libraries.

In [1]:
import numpy as np
import QCompute
import qcompute_qep.tomography as tomography
import qcompute_qep.quantum.pauli as pauli
import qcompute_qep.utils.circuit
import qcompute_qep.quantum.metrics as metrics

Then we set up the quantum program for CNOT gate.

In [2]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)

# Manually decompose the CNOT gate using the CZ gate, where CNOT: q1 -> q0
QCompute.H(qp.Q[0])
QCompute.CZ(qp.Q[1], qp.Q[0])
QCompute.H(qp.Q[0])


 
![QPT](./figures/qpt-CNOT-example.png "Figure 3: We take the CNOT gate as the target of QPT. ")
 
The circuit is shown in the figure above.
Before QPT, we compute numerically the ideal PTM of CNOT for reference.

In [ ]:
ideal_cnot = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_cnot).data
print(ideal_ptm)

Then we set the quantum computer (instance of QComputer). The QuantumComputer can be a simulator or a hardware interface. The rest is simple, we initialize a ProcessTomography instance, call the tomography procedure and obtain the noisy quantum state. Here, we set the method as 'inverse' meas that we estimate $[\mathcal{N}]$ with linear inversion.

In [ ]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian
# Initialize a ProcessTomography instance
st = tomography.ProcessTomography()
# Call the tomography procedure and obtain the noisy CZ gate
noisy_ptm = st.fit(qp, qc, prep_basis='Pauli', meas_basis='Pauli', method='inverse', shots=8192, ptm=True)


Finally, we can analyze the experimental data and visualize these PTMs to see the effect of QPT.

In [ ]:
print("****** The average gate fidelity between these two PTMs is: {}".format(
        metrics.average_gate_fidelity(ideal_ptm, noisy_ptm)))

# Visualize these PTMs
diff_ptm = ideal_ptm - noisy_ptm
tomography.compare_process_ptm(ptms=[ideal_ptm, noisy_ptm.data, diff_ptm],
                               show_labels=True)



![](./figures/qpt-output.png "Figure 4. We visualize the result of example.")


## Summary

This tutorial describes how to use Quantum Process Tomography method to deal with identifying an unknown quantum dynamical process on [Baidu Quantum Platform](https://quantum.baidu.com/).

## References
[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).